# Google Ads Keyword Ideas from URLs

This tutorial demonstrates how to fetch keyword ideas from Google Ads Keyword Planner for a list of URLs, aggregate the results, and highlight unique high-value keywords.

## Install dependencies

In [ ]:
!pip install google-ads plotly -q

## Authenticate with Google Ads
Fill in the path to your service account JSON file and the customer details from your Google Ads account.

In [ ]:

import pandas as pd
from google.oauth2 import service_account
from google.ads.googleads.client import GoogleAdsClient

SERVICE_ACCOUNT_FILE = '/path/to/service_account.json'  # update this
DEVELOPER_TOKEN = 'INSERT_DEVELOPER_TOKEN'
LOGIN_CUSTOMER_ID = 'INSERT_LOGIN_CUSTOMER_ID'
CUSTOMER_ID = 'INSERT_CUSTOMER_ID'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
client = GoogleAdsClient(credentials=credentials,
                         developer_token=DEVELOPER_TOKEN,
                         login_customer_id=LOGIN_CUSTOMER_ID)


## Helper function to fetch keyword ideas

In [ ]:

def get_keyword_ideas_from_url(page_url):
    keyword_plan_idea_service = client.get_service('KeywordPlanIdeaService')
    google_ads_service = client.get_service('GoogleAdsService')

    language_rn = google_ads_service.language_constant_path('1000')  # English

    request = client.get_type('GenerateKeywordIdeasRequest')
    request.customer_id = CUSTOMER_ID
    request.language = language_rn
    request.include_adult_keywords = False
    request.keyword_plan_network = client.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH_AND_PARTNERS
    request.url_seed.url = page_url

    response = keyword_plan_idea_service.generate_keyword_ideas(request=request)

    ideas = []
    comp_enum = client.enums.KeywordPlanCompetitionLevelEnum.KeywordPlanCompetitionLevel
    for idea in response.results:
        m = idea.keyword_idea_metrics
        ideas.append({
            'Keyword Text': idea.text,
            'Average Monthly Searches': m.avg_monthly_searches or 0,
            'Competition': comp_enum.Name(m.competition),
            'Competition Index': getattr(m, 'competition_index', 0),
            'URL': page_url
        })
    df = pd.DataFrame(ideas)
    return df.drop_duplicates(subset=['Keyword Text','URL']).reset_index(drop=True)


## Load URLs

In [ ]:

# CSV file should contain a column named 'URL'
urls_df = pd.read_csv('urls.csv')
all_urls = urls_df['URL'].dropna().astype(str).unique().tolist()


## Fetch keyword ideas for each URL

In [ ]:

keyword_dfs = []
for url in all_urls:
    print(f'Fetching keyword ideas for: {url}')
    try:
        df = get_keyword_ideas_from_url(url)
        if not df.empty:
            keyword_dfs.append(df)
    except Exception as e:
        print(f'  -> Error for {url}: {e}')

combined_df = pd.concat(keyword_dfs, ignore_index=True)
combined_df = combined_df.drop_duplicates(subset=['URL','Keyword Text']).reset_index(drop=True)

# keyword length
combined_df['KW Length'] = combined_df['Keyword Text'].str.split().str.len()
combined_df.head()


## Aggregate keywords across URLs

In [ ]:

kw_rep = combined_df.drop_duplicates(subset='Keyword Text').copy()
kw_count = combined_df.groupby('Keyword Text')['URL'].nunique().reset_index(name='URL Count')
kw_urls = combined_df.groupby('Keyword Text')['URL'].agg(list).reset_index(name='URLs')
keyword_crossurl = (
    kw_rep.merge(kw_count, on='Keyword Text').merge(kw_urls, on='Keyword Text')
)
keyword_crossurl.head()


## Filter for unique, high-value keywords

In [ ]:

filtered = keyword_crossurl[keyword_crossurl['URL Count'] <= 8].copy()


### Weighting formula

In [ ]:

import numpy as np

def weighting_formula(avg_monthly_searches, competition_index, url_count):
    return np.log(avg_monthly_searches + 1) * (1 / (competition_index + 1)) * (1 / (url_count + 1))

filtered['Score'] = filtered.apply(
    lambda row: weighting_formula(row['Average Monthly Searches'], row['Competition Index'], row['URL Count']),
    axis=1
)
filtered = filtered.sort_values('Score', ascending=False).reset_index(drop=True)
filtered.head()


## Visualize results

In [ ]:

import plotly.express as px

fig = px.scatter(filtered,
                 x='Average Monthly Searches',
                 y='Competition Index',
                 size='Score',
                 color='URL Count',
                 hover_data=['Keyword Text','Score'],
                 title='Keyword Opportunities')
fig.update_layout(xaxis_range=[0, filtered['Average Monthly Searches'].max()*1.1],
                  yaxis_range=[0, filtered['Competition Index'].max()*1.1])
fig.show()


## Save the results

In [ ]:

filtered.to_csv('keyword_opportunities.csv', index=False)
